# Evaluation Pipeline

Here, I am implementing the metrices used for evaluating the evaluation quality

In [ ]:
#imports 

import os
import sys

path_to_add = os.path.abspath(os.path.join(os.getcwd(), '..'))
if path_to_add not in sys.path:
    sys.path.append(path_to_add)

import time

from efficientnet.tfkeras import EfficientNetB4, EfficientNetB3
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from transformers import ViTImageProcessor, ViTForImageClassification, ViTFeatureExtractor, DetrForSegmentation, DetrFeatureExtractor
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry

import torch

from Utilities.utilities import *
from Utilities.lime_segmentation import *
from Utilities.xai_evaluation import *


In [ ]:
config = {
    
    "XAI_algorithm": {
        "DSEG" : True,
        "LIME" : True,
        "SLIME" : True,
        "BayesLime" : True,
        "GLIME" : True,
    },

    'computation':{
        'num_workers': 3,
        'gpu_device': False,
        'gpu_num': "1",
    },
    
    'model_to_explain':{
        'EfficientNet' : False,
        'ResNet': True,
        'VisionTransformer': False,
    },
    
    'lime_segmentation':{
        #Lime Parameters
        'num_samples': 256,
        'num_features': 1000,
        'min_weight': 0.01,
        'top_labels': 1,
        'hide_color': None,
        'batch_size': 10,
        'verbose': True,
        
        'slic': False,
        'quickshift': False,
        'felzenszwalb': True,
        'watershed': False,
        
        'all_dseg': False,
        'DETR' : False,
        'SAM' : True,
        'points_per_side' : 32,
        'min_size': 512,
        
        'fit_segmentation' : True,        
        'slic_compactness' : 16,
        'num_segments': 20,
        'markers' : 16,
        'kernel_size' : 6,
        'max_dist' : 32,
        
        #Lime Segmentation Parameters
        'iterations': 1,
        'shuffle': False,
        'max_segments': 8,
        'min_segments': 1,
        'auto_segment': False, 
        
        # LIME Explanation Parameters
        "num_features_explanation": 2,
        "adaptive_num_features": False,
        "adaptive_fraction": True,
        
        'hide_rest': True,
        'positive_only': True,
        
    },
    
    "evaluation": {
        'noisy_background': True,
        
        #Correctness
        "model_randomization" : True,
        "explanation_randomization" : True,
        
        "single_deletion": True,
        "fraction" : 0.1,
        "fraction_std" : 0.05,
        
        "incremental_deletion": True,
        "incremental_deletion_fraction": 0.15,
        
        "stability": True,
        "repetitions": 8,
        
        #Output Completeness
        "preservation_check": True,
        "deletion_check": True,
        
        #Consistency
        "variation_stability": True,
        
        #Contrastivity
        "target_discrimination": True,
        
        #Compactness
        "size": True,
    }
}

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = config['computation']['gpu_num'] if config['computation']['gpu_device'] else ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' if config['computation']['gpu_device'] else ''

os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda-11.4' if config['computation']['gpu_device'] else ''#-10.1' #--xla_gpu_cuda_data_dir=/usr/local/cuda, 
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 ,--tf_xla_enable_xla_devices' if config['computation']['gpu_device'] else ''#'--tf_xla_auto_jit=2' #, --tf_xla_enable_xla_devices
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

model_eff_b3 = EfficientNetB3(weights='imagenet')

if config['model_to_explain']['EfficientNet']: 
    model_explain = EfficientNetB4(weights='imagenet')
    model_explain_processor = None 
elif config['model_to_explain']['ResNet']:
    model_explain = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
    model_explain.eval()
    model_explain_processor = None
elif config['model_to_explain']['VisionTransformer']:
    model_explain_processor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch32-384')
    model_explain = ViTForImageClassification.from_pretrained('google/vit-base-patch32-384')

if config['lime_segmentation']['DETR']:
    feature_extractor = DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-50-panoptic")
    model = DetrForSegmentation.from_pretrained("facebook/detr-resnet-50-panoptic")

else:
    sam_checkpoint = "../Models/pretrained/sam_vit_h_4b8939.pth"
    model_type = "vit_h"
    model = sam_model_registry[model_type](checkpoint=sam_checkpoint)
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    if config['computation'].get('gpu_device'):
        model.to(device="cuda:0")
    else:
        model.to(device="cpu")

    mask_generator = SamAutomaticMaskGenerator(model,
                                            min_mask_region_area=config['lime_segmentation']['min_size'],
                                            points_per_side=config['lime_segmentation']['points_per_side'],
                                            )
   
    

In [ ]:
def evaluate_dataset_parallel_in_steps(model_eff, feature_extractor, dataset_path, model, model_test, config, steps = 5, date_path = None, model_explain_processor = None):
    # Get the list of sub-images in the dataset path
    sub_images_raw = os.listdir(dataset_path)
    
    if date_path == None:
        date_path = time.strftime("%y_%m_%d_%H_%M_%S")
        sub_images = sub_images_raw[0:steps]
        
        image_dictionary = {}
    
    else:
        
        if os.path.exists('./Dataset/Results/'+date_path+'/'"eval_results.pkl"):
            image_dictionary = pickle.load(open('./Dataset/Results/'+date_path+'/'"eval_results.pkl", "rb"))
        
        print(image_dictionary.keys())
        
        sub_images = []
        for i in sub_images_raw:
            if i not in image_dictionary.keys() and i != ".DS_Store":
                if steps > 0:
                    sub_images.append(i)
                    steps -= 1
                
    print(sub_images)
    if len(sub_images) > 0:

        # Create the 'Results' directory if it doesn't exist
        pathlib.Path('./Dataset/Results/'+date_path).mkdir(parents=True, exist_ok=True)
        json.dump(config, open('./Dataset/Results/'+date_path+'/config.json', 'w'))
        # Create a Pool with the specified number of processes
        parallel_data_generation = Parallel(n_jobs=config['computation']['num_workers'], verbose=3, backend='loky') #loky #sequential multiprocessing
        results = parallel_data_generation(delayed(evaluate_sub_image)(sub_image, dataset_path, date_path, model_eff, feature_extractor, model, model_test, config, model_explain_processor = model_explain_processor) for sub_image in sub_images)


        # Populate the image dictionary with results
        for sub_image, result in results:
            image_dictionary[sub_image] = result

        pickle.dump(image_dictionary, open('./Dataset/Results/'+date_path+'/'"eval_results.pkl", "wb"))
        
        # Return the image dictionary
        return dataset_path
    else:
        print("error")
        return None
    

def evaluate_stable_dataset_parallel_in_steps(model_eff, mask_generator, dataset_path, model_sam, model_eff_b3, config, model_explain_processor = None, date_path = None, steps = 3, num_cpus=None):
    
    if date_path == None:
        date_path = time.strftime("%y_%m_%d_%H_%M_%S")

    for i in range(4):
        try:
            result = evaluate_dataset_parallel_in_steps(model_eff, mask_generator, dataset_path, model_sam, model_eff_b3, config, steps = 3, model_explain_processor = model_explain_processor,date_path = date_path)
        except:
            print("Error")
            continue
        

In [ ]:
dataset_path = "../Dataset/Evaluation/"

data_path = evaluate_dataset_parallel_in_steps(model_explain, mask_generator, dataset_path, model, model_eff_b3, config, steps = 3, model_explain_processor = model_explain_processor)

In [ ]:
dataset_path = "../Dataset/Evaluation/"
for i in range(6):
    data_path = evaluate_dataset_parallel_in_steps(model_explain, mask_generator, dataset_path, model, model_eff_b3, config, steps = 3, model_explain_processor = model_explain_processor, date_path = "24_02_22_08_57_17")